# **INSURANCE ANALITYCS: TRAVEL INSURANCE CLAIM**

**CREATED BY: PRIMA ADE SUKRONO**

## **BUSINESS PROBLEM UNDERSTANDING**

<u>**Context**</u>

Asuransi perjalanan adalah jenis asuransi yang memberikan perlindungan selama kita bepergian, baik di dalam negeri maupun luar negeri. Beberapa negara bahkan mewajibkan wisatawan untuk memiliki asuransi perjalanan, misalnya negara-negara di Eropa dan Amerika. Besarnya premi bergantung pada cakupan yang diinginkan, lama perjalanan, dan tujuan perjalanan. Perusahaan asuransi perjalanan ingin mengetahui pemegang polis yang akan mengajukan klaim asuransi. Data pemegang polis di perusahaan asuransi merupakan data historis yang terdiri dari destinasi, produk asuransi, dan sebagainya.

Target:
- 0: Tidak mengajukan claim (No)
- 1: Mengajukan claim (Yes)

---

<u>**Problem Statement**</u>

Prediksi risiko secara manual masih dapat dilakukan oleh tim Underwriting, namun proses ini cenderung memakan waktu lebih lama dan memiliki potensi kesalahan yang lebih tinggi. Keterbatasan dalam memproses kombinasi fitur secara kompleks secara manual dapat menyebabkan kesalahan prediksi—khususnya `False Negative`, yaitu calon nasabah dengan risiko tinggi klaim yang tidak terdeteksi.

Jika pendekatan konservatif digunakan, di mana semua calon nasabah diasumsikan akan mengajukan klaim, maka akan terjadi ketidakadilan dalam penetapan premi. Nasabah berisiko rendah diperlakukan sama dengan yang berisiko tinggi, yang dapat menurunkan minat pembelian polis baru dan mendorong nasabah loyal untuk berpindah ke kompetitor yang menawarkan skema premi lebih adil.

Untuk meningkatkan efisiensi operasional dan profitabilitas, tim Underwriting memerlukan sistem prediktif berbasis machine learning yang mampu mengidentifikasi calon nasabah dengan risiko klaim tinggi secara lebih akurat, serta memberikan wawasan terhadap faktor-faktor utama yang memengaruhi risiko tersebut. Sistem ini diharapkan mendukung implementasi strategi premi berbasis resiko (risk-based pricing), sehingga penetapan premi dapat lebih tepat sasaran, mengurangi kerugian finansial akibat klaim tak terduga, dan memperkuat daya saing perusahaan di pasar asuransi.

---

<u>**Tujuan Analisis**</u>

1. Memprediksi risiko klaim setiap calon pemegang polis berdasarkan data historis seperti durasi perjalanan, produk asuransi, tujuan perjalanan, usia, dan lainnya.
2. Mengidentifikasi faktor-faktor utama (risk drivers) yang menyebabkan nasabah cenderung mengajukan klaim, sebagai dasar untuk:
   - Penyesuaian premi individu (risk-based pricing)
   - Desain ulang produk dan strategi distribusi
   - Segmentasi pelanggan
3. Mendukung strategi penetapan harga yang adaptif dengan mempertimbangkan tingkat risiko yang diprediksi oleh model.
4. Membantu tim underwriting dan tim terkait membuat keputusan yang lebih akurat terhadap nasabah berisiko tinggi melalui rekomendasi berbasis data.

--- 

<u>**Pendekatan Analitik**</u>

Kita akan melakukan analisa dari data historis nasabah untuk mengidentifikasi pola yang membedakan antara nasabah yang mengajukan klaim dan yang tidak, lalu membangun model machine learning untuk memprediksi risiko klaim secara otomatis.

<u>**Skenario Awal**</u>

Berdasarkan data awal, jika diasumsikan prediksi claim hanya berdasarkan durasi perjalanan lebih dari 30 hari:

Maka kerugian yang dapat ditimbulkan adalah:
- FN 305
- FP 100

Cost ?????????

<u>**Metric Evaluasi**</u>

Jika dilihat dari sisi biaya yang ditimbulkan oleh kesalahan prediksi (*bukan total biaya keseluruhan*), maka kesalahan dalam **tidak mendeteksi nasabah yang sebenarnya mengajukan klaim (FN) memiliki dampak biaya yang jauh lebih besar** dibandingkan dengan salah memprediksi nasabah yang sebenarnya tidak mengajukan klaim (FP).

Berdasarkan pertimbangan tersebut, model yang dibangun akan difokuskan untuk semaksimal mungkin mendeteksi nasabah yang benar-benar akan mengajukan klaim (TP), serta meminimalkan kesalahan prediksi pada nasabah yang berisiko tinggi untuk claim (FN), tanpa sepenuhnya mengabaikan tingkat kesalahan dalam memprediksi nasabah yang tidak mengajukan claim (FP).

Oleh karena itu, metrik evaluasi utama yang akan digunakan adalah **Recall**, karena recall mencerminkan kemampuan model dalam menangkap sebanyak mungkin kasus klaim yang benar.

## **DATA UNDERSTANDING & CLEANING**

In [1]:
# Import libraries

# General libraries
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import plotly.express as px

# data splitting
from sklearn.model_selection import train_test_split

# impute missing values
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer

# preprocessing
from sklearn.preprocessing import MinMaxScaler, StandardScaler, RobustScaler
from sklearn.preprocessing import OneHotEncoder
from category_encoders import OrdinalEncoder, BinaryEncoder
from sklearn.preprocessing import FunctionTransformer

# column transformer & pipeline
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from imblearn.pipeline import Pipeline

# cross validation
from sklearn.model_selection import cross_val_score

# algorithms
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier

# evaluation metric
from sklearn.metrics import precision_score, recall_score, f1_score, confusion_matrix, classification_report

# Gridsearch
from sklearn.model_selection import GridSearchCV

### 1. Load Dataset

In [2]:
# Import dataset
df_travel = pd.read_csv('data_travel_insurance.csv')

### 2. Data Understanding

In [3]:
# Lihat 5 data teratas
df_travel.head()

,Agency,Agency Type,Distribution Channel,Product Name,Gender,Duration,Destination,Net Sales,Commision (in value),Age,Claim
0,C2B,Airlines,Online,Annual Silver Plan,F,365,SINGAPORE,216.0,54.0,57,No
1,EPX,Travel Agency,Online,Cancellation Plan,NaN,4,MALAYSIA,10.0,0.0,33,No
2,JZI,Airlines,Online,Basic Plan,M,19,INDIA,22.0,7.7,26,No
3,EPX,Travel Agency,Online,2 way Comprehensive Plan,NaN,20,UNITED STATES,112.0,0.0,59,No
4,C2B,Airlines,Online,Bronze Plan,M,8,SINGAPORE,16.0,4.0,28,No


Setiap baris dalam dataset merepresentasikan satu transaksi penjualan polis asuransi perjalanan, di mana produk asuransi yang sama dapat terjual berkali-kali kepada nasabah yang berbeda atau untuk perjalanan yang berbeda.


Penjelasan setiap kolom dalam dataset:

- Agency: Nama dari agency.
- Agency Type: Jenis dari agen asuransi perjalanan.
- Distribution Channel: Channel distribusi dari agen asuransi perjalanan.
- Product Name: Nama dari produk asuransi perjalanan.
- Gender: Jenis kelamin dari pemegang polis.
- Duration: Lama waktu perjalanan. (*karena tidak disebutkan sebelumnya, maka diasumsikan ini dalam bentuk hari*)
- Destination: Tujuan perjalanan.
- Net Sales: Jumlah penjualan bersih dari polis asuransi perjalanan.
- Commission (in value): Komisi yang diterima agen dari hasil penjualan.
- Age: Umur dari pemegang polis.
- Claim: Status Claim.

In [4]:
# Lihat info data
df_travel.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 44328 entries, 0 to 44327
Data columns (total 11 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   Agency                44328 non-null  object 
 1   Agency Type           44328 non-null  object 
 2   Distribution Channel  44328 non-null  object 
 3   Product Name          44328 non-null  object 
 4   Gender                12681 non-null  object 
 5   Duration              44328 non-null  int64  
 6   Destination           44328 non-null  object 
 7   Net Sales             44328 non-null  float64
 8   Commision (in value)  44328 non-null  float64
 9   Age                   44328 non-null  int64  
 10  Claim                 44328 non-null  object 
dtypes: float64(2), int64(2), object(7)
memory usage: 3.7+ MB


Total data ada 44,328 baris dan 11 kolom, dimana terlihat ada 1 kolom terdapat missing value yaitu `Gender`, dan Type data terlihat sudah sesuai dengan isi kolom.

In [5]:
# Lihat Deskripsi data
df_travel.describe(include='all')

,Agency,Agency Type,Distribution Channel,Product Name,Gender,Duration,Destination,Net Sales,Commision (in value),Age,Claim
count,44328,44328,44328,44328,12681,44328.000000,44328,44328.000000,44328.000000,44328.000000,44328
unique,16,2,2,26,2,NaN,138,NaN,NaN,NaN,2
top,EPX,Travel Agency,Online,Cancellation Plan,M,NaN,SINGAPORE,NaN,NaN,NaN,No
freq,24656,32113,43572,12979,6504,NaN,9267,NaN,NaN,NaN,43651
mean,NaN,NaN,NaN,NaN,NaN,49.424292,NaN,40.550948,9.707692,39.925600,NaN
std,NaN,NaN,NaN,NaN,NaN,109.153961,NaN,48.661970,19.625637,13.954926,NaN
min,NaN,NaN,NaN,NaN,NaN,-1.000000,NaN,-357.500000,0.000000,0.000000,NaN
25%,NaN,NaN,NaN,NaN,NaN,9.000000,NaN,18.000000,0.000000,35.000000,NaN
50%,NaN,NaN,NaN,NaN,NaN,22.000000,NaN,26.500000,0.000000,36.000000,NaN
75%,NaN,NaN,NaN,NaN,NaN,53.000000,NaN,48.000000,11.550000,43.000000,NaN


Dari hasil deskripsi data diatas, ditemukan bahwa terdapat anomali pada isi dari beberapa kolom berikut:
- Duration: Minimum hari adalah -1
- Net Sales: Minimum penjualannya -357.5
- Commision: Minimum dan median-nya 0
- Age: Minimumnya 0 dan Maximumnya 118

Maka perlu dilakukan pengecekan lebih lanjut pada kolom-kolom tersebut pada tahap pengecekan missing value.

In [6]:
# Lihat proporsi data
df_travel['Claim'].value_counts(normalize=True)

Claim
No     0.984727
Yes    0.015273
Name: proportion, dtype: float64

Proporsi data terlihat imbalance, dimana 98% data tersebar pada nasabah yang tidak claim (No), sedangkan hanya 2% persen data pada nasabah yang claim (Yes). Perlu dilakukan percobaan balancing data pada tahap imbalance treatment.

Kesimpulan dari data understanding ini adalah:
1. Setiap baris dalam dataset merepresentasikan satu transaksi penjualan polis asuransi perjalanan.
2. Terdapat 1 Kolom yang beberapa nilainya hilang, yaitu kolom `Gender`
3. Terdapat 4 kolom yang memiliki anomali pada isinya yang perlu dilakukan pengecekan lebih lanjut, yaitu kolom `Duration`, `Net Sales`, `Commission`, dan `Age`.
4. Proporsi data terlihat imbalance, dimana 98% tersebar pada target 'No' dan hanya 2% tersebar pada target 'Yes'. Perlu dilakukan percobaan imbalance treatment untuk membandingkan performa sebelum dan setelah balancing.

### 3. Data Cleaning

#### A. Missing Value

In [7]:
# Lihat missing value dan anomali pada data
list_item = []
num_col = df_travel.select_dtypes(include='number').columns

for col in df_travel.columns :
    item = [col, df_travel[col].isna().sum(), round((df_travel[col].isna().sum()/len(df_travel[col])) * 100, 2)]
    if col in num_col:
        item.extend([df_travel[col].min(),
                    df_travel[col].median(),
                    df_travel[col].max()])
    else:
        item.extend(['Nan', 'Nan', 'Nan'])
    list_item.append(item)

df_info = pd.DataFrame(data=list_item, columns=['features', 'null', 'null_perc', 'min', 'Median', 'Max'])
df_info

,features,null,null_perc,min,Median,Max
0,Agency,0,0.00,Nan,Nan,Nan
1,Agency Type,0,0.00,Nan,Nan,Nan
2,Distribution Channel,0,0.00,Nan,Nan,Nan
3,Product Name,0,0.00,Nan,Nan,Nan
4,Gender,31647,71.39,Nan,Nan,Nan
5,Duration,0,0.00,-1,22.0,4881
6,Destination,0,0.00,Nan,Nan,Nan
7,Net Sales,0,0.00,-357.5,26.5,810.0
8,Commision (in value),0,0.00,0.0,0.0,283.5
9,Age,0,0.00,0,36.0,118


Melanjutkan hasil data understanding sebelumnya, terlihat memang ada nilai null dan anomlai pada beberapa fitur. Kita akan tangani nilai null dan anomali satu persatu.

Yang pertama, kita akan coba tangani nilai yang hilang pada fitur `Gender`.

Dari informasi diatas kita bisa persentase nilai hilangnya hampir mencapai 3/4 dari total datanya (71.39%). Jika diteliti lebih lanjut, tidak ada fitur lain yang bisa merepresentasikan fitur`Gender` ini sebagai referensi pengisiannya, dan jika kita paksakan isi dengan 'modus' atau 'unknown' akan menjadi bias dalam prediksi karena tidak ada jenis kelamin yang 'unknown' di dunia nyata.

Maka dari itu, fitur ini akan kita hapus karena selain persentase hilangnya yang begitu banyak dan susah untuk menebak isinya, fitur ini juga dirasa tidak akan berpengaruh secara signifikan terhadap keputusan nasabah untuk claim jika terjadi sesuatu dalam perjalanannya.

**`Gender`**

In [8]:
# Menghapus fitur Gender
df_travel = df_travel.drop(columns=['Gender'])
df_travel.columns

Index(['Agency', 'Agency Type', 'Distribution Channel', 'Product Name',
       'Duration', 'Destination', 'Net Sales', 'Commision (in value)', 'Age',
       'Claim'],
      dtype='object')

Kita sudah berhasil menghapus fitur `Gender`, selanjutnya kita akan coba lihat anomali pada fitur duration.

**`Duration`**

In [9]:
# Melihat anomali pada data dengan Duration < 0
df_travel[df_travel['Duration'] < 0]

,Agency,Agency Type,Distribution Channel,Product Name,Duration,Destination,Net Sales,Commision (in value),Age,Claim
26228,JZI,Airlines,Online,Basic Plan,-1,MALAYSIA,18.0,6.3,118,No
26494,JZI,Airlines,Online,Basic Plan,-1,BRUNEI DARUSSALAM,18.0,6.3,118,No


Ternyata hanya 2 transaksi yang nilainya '-1', dimana keduanya berasal dari Agensi dan produk yang sama dan hanya dibedakan oleh tujuannya saja. Age disini juga mempunyai nilai yang anomali yaitu 118. Maka dari itu, data ini kemungkinan misstyping.

Kita akan coba lihat apakah ada kemiripan kedua data duration tersebut dengan transaksi lainnnya dengan tujuan yang sama.

In [10]:
# Mengambil data dengan agensi 'JZI' dengan tujuan 'MALAYSIA' dan 'BRUNEI DARUSSALAM'
malaysia = df_travel[(df_travel['Agency'] == 'JZI') & (df_travel['Destination'] == 'MALAYSIA')].sort_values(by='Duration').head()
brunei = df_travel[(df_travel['Agency'] == 'JZI') & (df_travel['Destination'] == 'BRUNEI DARUSSALAM')].sort_values(by='Duration').head()

# Menggabungkan kedua data untuk dilihat persamaannya
concat = pd.concat([malaysia, brunei])
concat

,Agency,Agency Type,Distribution Channel,Product Name,Duration,Destination,Net Sales,Commision (in value),Age,Claim
26228,JZI,Airlines,Online,Basic Plan,-1,MALAYSIA,18.0,6.3,118,No
7359,JZI,Airlines,Online,Basic Plan,1,MALAYSIA,18.0,6.3,50,No
38217,JZI,Airlines,Online,Basic Plan,1,MALAYSIA,18.0,6.3,54,No
37046,JZI,Airlines,Online,Basic Plan,1,MALAYSIA,-18.0,6.3,47,No
24093,JZI,Airlines,Online,Basic Plan,1,MALAYSIA,18.0,6.3,42,No
26494,JZI,Airlines,Online,Basic Plan,-1,BRUNEI DARUSSALAM,18.0,6.3,118,No
8014,JZI,Airlines,Online,Basic Plan,3,BRUNEI DARUSSALAM,18.0,6.3,36,No
35707,JZI,Airlines,Online,Basic Plan,3,BRUNEI DARUSSALAM,18.0,6.3,48,No
44231,JZI,Airlines,Online,Basic Plan,4,BRUNEI DARUSSALAM,18.0,6.3,43,No
6740,JZI,Airlines,Online,Basic Plan,5,BRUNEI DARUSSALAM,18.0,6.3,27,No


Untuk yang tujuannya 'MALAYSIA' datanya mirip dengan transaksi lainnya yang durasi perjalanannya 1 hari, maka dari itu kita akan ubah nilainya dari -1 ke 1.

Sedangkan tujuan 'BRUNEI DARUSSALAM' tidak ada yang durasi perjalanannya 1 hari, tapi bisa kita lihat bahwa harga penjualannya sama dari rentang 3-5 hari. Kemungkinan besar ini adalah harga bawah yang ditetapkan oleh agen/perusahaan asuransi perjalanan. Maka dari itu, agar nilainya tetap bervariasi namun tidak jauh dari nilai lainnya, kita akan rubah nilainya dari -1 ke 1.

Karena keduanya akan diubah ke nilai yang sama, maka kita akan langsung gunakan `.replace()` pada fungsi pandas.

In [11]:
# Mengganti data yang minus menjadi plus
df_travel['Duration'] = df_travel['Duration'].replace(-1, 1)
df_travel['Duration'].min()

np.int64(0)

Nlai '-1' sudah berhasil ditangani, namun nilai minimal sekarang adalah '0', kita perlu melihat apa artinya 0 ini, karena ini sebuah anomali jika nasabah membeli produk asuransi perjalanan namun tidak melakukan perjalanan.

In [12]:
# Melihat data dengan Duration = 0
df_travel[df_travel['Duration'] == 0]

,Agency,Agency Type,Distribution Channel,Product Name,Duration,Destination,Net Sales,Commision (in value),Age,Claim
242,JZI,Airlines,Online,Basic Plan,0,CHINA,22.00,7.70,34,No
1782,SSI,Airlines,Online,Ticket Protector,0,SINGAPORE,1.80,0.50,48,No
1853,JWT,Airlines,Online,Value Plan,0,INDIA,62.00,24.80,118,No
2558,SSI,Airlines,Online,Ticket Protector,0,SINGAPORE,9.77,2.74,48,No
3236,LWC,Travel Agency,Online,Single Trip Travel Protect Gold,0,INDIA,35.25,22.91,24,No
3406,SSI,Airlines,Online,Ticket Protector,0,SINGAPORE,2.86,0.80,48,No
3549,JZI,Airlines,Online,Basic Plan,0,VIET NAM,18.00,6.30,58,No
3591,JZI,Airlines,Online,Basic Plan,0,MYANMAR,18.00,6.30,26,No
3753,JWT,Airlines,Online,Value Plan,0,INDIA,31.00,12.40,118,No
4939,CWT,Travel Agency,Online,Rental Vehicle Excess Insurance,0,UNITED STATES,9.90,5.94,59,No


Total 47 baris data memang terlihat anomali dimana destinasi sudah ditentukan dan produk sudah terjual, namun tidak ada durasi perjalanannya.

Dari pemahaman data, nilai ini bisa saja direpresentasikan oleh produk yang dijual `Product Name`, tujuan perjalanan `Destination`, dan juga harga penjualannya `Net Sales`. Dari beberapa gabungan fitur tersebut kita bisa memperkirakan berapa hari biasanya perjalanan yang dilakukan untuk pembelian produk tersebut. Namun yang perlu diperhatikan bahwa bisa saja semua fitur tersebut sama pada rentang hari perjalanan yang ditentukan, seperti kasus sebelumnya pada tujuan 'BRUNEI DARUSSALAM'. 

Maka dari itu kita akan coba ambil rata-ratanya saja yang akan kita bulatkan sehingga bisa lebih merepresentasikan perkiraan dari rentangnya. Kita akan coba isikan nilai tersebut ke fitur Duration.

Untuk teknis pengisiannya kita akan menggunakan 'FunctionTransformer' dari library 'scikit-learn', dimana kita membuat function yang dapat mengisi nilai yang hilang dengan referensi fitur yang berkaitan. Function ini kemudian dimasukkan kedalam `FunctionTransformer` yang kemudian dimasukkan kedalam pipeline.

In [13]:
# Membuat fungsi untuk mengisi nilai 0 fitur Duration
def fill_duration(X):
    X = X.copy()

    # Buat group mean hanya dari baris yang Duration != 0dan net sales != 0
    ref_mean = (
        X[(X['Duration'] != 0) & (X['Net Sales'] != 0)]
        .groupby(['Product Name','Destination', 'Net Sales'])['Duration']
        .mean()
        .astype('int')
    )

    # Isi nilai yang null pada fitur Duration dengan group mean
    X['Duration'] = X.apply(
        lambda row: ref_mean.get((row['Product Name'], row['Destination'], row['Net Sales']), row['Duration'])
        if row['Duration'] == 0 else row['Duration'],
        axis=1
    )

    return X

# Membuat FunctionTransformer
duration_imputer = FunctionTransformer(fill_duration, validate=False)

In [14]:
# Copy file agar file original tidak terkontaminasi
df_travel_copy = df_travel.copy()

# Mencoba fungsi apakah sudah sesuai sebelum dimasukkan dalam pipeline
df_travel_copy = duration_imputer.fit_transform(df_travel)
df_travel_copy[df_travel_copy['Duration'] == 0]

,Agency,Agency Type,Distribution Channel,Product Name,Duration,Destination,Net Sales,Commision (in value),Age,Claim
3236,LWC,Travel Agency,Online,Single Trip Travel Protect Gold,0,INDIA,35.25,22.91,24,No
8344,C2B,Airlines,Online,Bronze Plan,0,SINGAPORE,-14.40,3.60,49,No
21249,C2B,Airlines,Online,Bronze Plan,0,SINGAPORE,14.40,3.60,49,No


Ternyata dari logika function yang sudah dibuat masih menyisakan 3 nilai null, yang berarti nilainya memang tidak tersedia pada kombinasi fitur produk, tujuan, dan penjualannya. Karena datanya sedikit dan agar tidak mempengaruhi akurasi model nantinya, maka kita hapus saja data ini dari dataframe originalnya.

In [15]:
# Menghapus data dengan durasi 0 pada product name = 'Single Trip Travel Protect Gold', dengan tujuan = 'INDIA' dan net sales = 35.25
df_travel = df_travel.drop(df_travel[(df_travel['Product Name'] == 'Single Trip Travel Protect Gold') & (df_travel['Destination'] == 'INDIA') & (df_travel['Net Sales'] == 35.25)].index)

# Menghapus data dengan durasi 0 pada product name = 'Bronze Plan', dengan tujuan = 'SINGAPORE', dan net sales = -14.40 & 14.4
df_travel = df_travel.drop(df_travel[(df_travel['Product Name'] == 'Bronze Plan') & (df_travel['Destination'] == 'SINGAPORE') & (df_travel['Net Sales'].isin([-14.4, 14.4]))].index)

In [16]:
# mencoba kembali fungsi apakah data dengan durasi 0 sudah tidak ada lagi
df_travel_copy = duration_imputer.fit_transform(df_travel)
display(df_travel_copy[df_travel_copy['Duration'] == 0], df_travel_copy.shape)
print("Jumlah `Duration` yang masih 0:", (df_travel_copy['Duration'] == 0).sum())

,Agency,Agency Type,Distribution Channel,Product Name,Duration,Destination,Net Sales,Commision (in value),Age,Claim


(44325, 10)

Jumlah `Duration` yang masih 0: 0


Terlihat sisa baris data menjadi 44,325 dari 44,328 baris sebelumnya, yang berarti data dengan durasi = 0 sisanya berhasil kita excludekan. Kita akan gunakan function ini pada pipeline nantinya agar proses imputasi bisa masuk dalam tahap machine learning yang bersih dan aman dari leakage.

Selanjutnya kita akan coba tangani anomali pada fitur `Net Sales`.

**`Net Sales`**

In [17]:
# melihat data dengan Net Sales < 0
display(df_travel[df_travel['Net Sales'] < 0])
display(df_travel[df_travel['Net Sales'] < 0]['Net Sales'].count())

,Agency,Agency Type,Distribution Channel,Product Name,Duration,Destination,Net Sales,Commision (in value),Age,Claim
94,C2B,Airlines,Online,Annual Silver Plan,365,SINGAPORE,-216.75,54.19,36,No
116,CWT,Travel Agency,Online,Rental Vehicle Excess Insurance,430,AUSTRALIA,-59.40,35.64,41,No
121,CWT,Travel Agency,Online,Rental Vehicle Excess Insurance,77,JAPAN,-29.70,17.82,59,No
199,EPX,Travel Agency,Online,Cancellation Plan,29,HONG KONG,-12.00,0.00,36,No
241,CWT,Travel Agency,Online,Rental Vehicle Excess Insurance,57,AUSTRALIA,-59.40,35.64,28,No
...,...,...,...,...,...,...,...,...,...,...
43694,EPX,Travel Agency,Online,2 way Comprehensive Plan,140,UNITED STATES,-98.00,0.00,29,No
43743,CWT,Travel Agency,Online,Rental Vehicle Excess Insurance,23,MALAYSIA,-29.70,17.82,29,No
43751,EPX,Travel Agency,Online,2 way Comprehensive Plan,147,SPAIN,-68.00,0.00,36,No
43858,EPX,Travel Agency,Online,Cancellation Plan,49,UNITED KINGDOM,-68.00,0.00,36,No


np.int64(482)

Terdapat 482 nilai minus dari fitur ini. Kita akan coba lihat salah satu agency apakah terdapat nilai yang sama dengan nilai minus ini pada produk yang sama.

In [18]:
# Melihat penjualan produk dari agensi 'C2B' dengan nama produk 'Annual Silver Plan' dan komisi = 54.19
df_travel[(df_travel['Agency'] == 'C2B') & (df_travel['Product Name'] == 'Annual Silver Plan') & (df_travel['Commision (in value)'] == 54.19)]

,Agency,Agency Type,Distribution Channel,Product Name,Duration,Destination,Net Sales,Commision (in value),Age,Claim
94,C2B,Airlines,Online,Annual Silver Plan,365,SINGAPORE,-216.75,54.19,36,No
8282,C2B,Airlines,Online,Annual Silver Plan,365,SINGAPORE,216.75,54.19,49,No
13810,C2B,Airlines,Online,Annual Silver Plan,388,SINGAPORE,216.75,54.19,36,No
14403,C2B,Airlines,Online,Annual Silver Plan,366,SINGAPORE,216.75,54.19,73,No
16163,C2B,Airlines,Online,Annual Silver Plan,370,SINGAPORE,216.75,54.19,78,No
21551,C2B,Airlines,Online,Annual Silver Plan,740,SINGAPORE,0.00,54.19,36,No
26229,C2B,Airlines,Online,Annual Silver Plan,365,SINGAPORE,216.75,54.19,48,No
32220,C2B,Airlines,Online,Annual Silver Plan,366,SINGAPORE,216.75,54.19,25,No
32995,C2B,Airlines,Online,Annual Silver Plan,403,SINGAPORE,216.75,54.19,47,No
38223,C2B,Airlines,Online,Annual Silver Plan,395,SINGAPORE,216.75,54.19,50,No


Benar saja, terlihat bahwa pada produk dan komisi yang sama, Net Sales-nya juga sama. Ini mengindikasikan bahwa kemungkinan nilai minus adalah kesalahan dalam penginputan.

Maka dari itu, untuk semua nilai minus akan kita absolutkan saja supaya jadi positif, karena terlihat nilainya sudah benar hanya salah input.

In [19]:
# Mengubah nilai minus dari fitur Net Sales menjadi absolut (plus)
df_travel['Net Sales'] = df_travel['Net Sales'].abs()
display(df_travel['Net Sales'].min(), df_travel[df_travel['Net Sales']==0], df_travel[df_travel['Net Sales'] == 0]['Net Sales'].count())

np.float64(0.0)

,Agency,Agency Type,Distribution Channel,Product Name,Duration,Destination,Net Sales,Commision (in value),Age,Claim
45,CWT,Travel Agency,Online,Rental Vehicle Excess Insurance,24,UNITED ARAB EMIRATES,0.0,23.76,27,No
60,CWT,Travel Agency,Online,Rental Vehicle Excess Insurance,14,AUSTRALIA,0.0,17.82,31,No
63,EPX,Travel Agency,Online,Cancellation Plan,100,UNITED STATES,0.0,0.00,37,No
75,C2B,Airlines,Online,Silver Plan,23,SINGAPORE,0.0,5.63,25,No
78,EPX,Travel Agency,Online,Cancellation Plan,73,BELGIUM,0.0,0.00,36,No
...,...,...,...,...,...,...,...,...,...,...
44207,EPX,Travel Agency,Online,2 way Comprehensive Plan,152,UNITED STATES,0.0,0.00,49,No
44213,CCR,Travel Agency,Offline,Comprehensive Plan,56,THAILAND,0.0,9.57,20,No
44268,JZI,Airlines,Online,Basic Plan,8,HONG KONG,0.0,12.25,69,No
44273,EPX,Travel Agency,Online,Cancellation Plan,52,PHILIPPINES,0.0,0.00,36,No


np.int64(1291)

Oke, kita sudah berhasil merubah nilai minus, tapi masih menyisakan nilai anomali yaitu 0. Dimana nilai ini terdeteksi ada 1,291 data.

Nilai ini dianggap anomali karena seharusnya produk yang sudah di beli mempunyai harga penjualan, namun harga 0 disini mengindikasikan harga penjualan tidak berhasil diinput dengan benar. Oleh karena itu, kita coba imputasi nilai 0 tersebut dengan pendekatan yang mirip dengan sebelumnya. Namun kali ini kita akan memasukkan nilai originalnya dari group yang sama, bukan rata-ratanya. Karena terlihat nilai `'Net Sales'` berkaitan dengan produk dan tujuan yang sama, maka fitur yang akan dijadikan referensi kali ini adalah `Product Name` dan `Destination`, namun hanya pada data yang nilai komisinya tidak kosong agar tidak bias.

Untuk fitur `Net Sales` yang bertipe data float, kita akan ganti nilai 0 nya menjadi Null/Nan agar lebih memudahkan dalam imputasi jika harus dilanjutkan oleh metode imputasi lain. Hal ini tidak dilakukan pada imputasi fitur `Duration` sebelumnya karena nilai Nan hanya bisa diterima oleh kolom bertipe data float.

In [20]:
# Merubah nilai 0 menjadi Nan
df_travel['Net Sales'] = df_travel['Net Sales'].replace(0, np.nan)

In [21]:
# melihat data dengan Net Sales yang null
df_travel[df_travel['Net Sales'].isnull()]

,Agency,Agency Type,Distribution Channel,Product Name,Duration,Destination,Net Sales,Commision (in value),Age,Claim
45,CWT,Travel Agency,Online,Rental Vehicle Excess Insurance,24,UNITED ARAB EMIRATES,NaN,23.76,27,No
60,CWT,Travel Agency,Online,Rental Vehicle Excess Insurance,14,AUSTRALIA,NaN,17.82,31,No
63,EPX,Travel Agency,Online,Cancellation Plan,100,UNITED STATES,NaN,0.00,37,No
75,C2B,Airlines,Online,Silver Plan,23,SINGAPORE,NaN,5.63,25,No
78,EPX,Travel Agency,Online,Cancellation Plan,73,BELGIUM,NaN,0.00,36,No
...,...,...,...,...,...,...,...,...,...,...
44207,EPX,Travel Agency,Online,2 way Comprehensive Plan,152,UNITED STATES,NaN,0.00,49,No
44213,CCR,Travel Agency,Offline,Comprehensive Plan,56,THAILAND,NaN,9.57,20,No
44268,JZI,Airlines,Online,Basic Plan,8,HONG KONG,NaN,12.25,69,No
44273,EPX,Travel Agency,Online,Cancellation Plan,52,PHILIPPINES,NaN,0.00,36,No


In [22]:
# Mendefinisikan Fungsi untuk mengisi Net Sales yang kosong
def fill_netsales(X):
    X = X.copy()

    # Buat referensi nilai dari baris yang net salesnya tidak mengandung nilai Nan
    ref_mean = (
        X[X['Net Sales'].notnull()]
        .drop_duplicates(['Product Name', 'Destination'])
        .set_index(['Product Name', 'Destination'])['Net Sales']
        .to_dict()
    )

    # Isi nilai Net Sales yang null dengan referensi dari group produk dan tujuan
    X['Net Sales'] = X.apply(
        lambda row: ref_mean.get((row['Product Name'], row['Destination']), row['Net Sales'])
        if pd.isnull(row['Net Sales']) else row['Net Sales'],
        axis=1
    )

    return X

# Membuat FunctionTransformer
netsales_imputer = FunctionTransformer(fill_netsales, validate=False)

In [23]:
# Mencoba fungsi apakah data yang hilang pada fitur Net Sales sudah tidak ada lagi
df_travel_copy = netsales_imputer.fit_transform(df_travel)
display(df_travel_copy[df_travel_copy['Net Sales'].isnull()])
print("Jumlah `Net Sales` yang masih Nan:", df_travel_copy['Net Sales'].isnull().sum())

,Agency,Agency Type,Distribution Channel,Product Name,Duration,Destination,Net Sales,Commision (in value),Age,Claim
7128,LWC,Travel Agency,Online,Single Trip Travel Protect Platinum,2,ANGOLA,NaN,16.00,19,No
13098,EPX,Travel Agency,Online,1 way Comprehensive Plan,8,BAHRAIN,NaN,0.00,36,No
17786,CWT,Travel Agency,Online,Rental Vehicle Excess Insurance,14,RUSSIAN FEDERATION,NaN,17.82,36,No
17879,TST,Travel Agency,Offline,Travel Cruise Protect Family,30,MALAYSIA,NaN,22.75,32,No
18028,CWT,Travel Agency,Online,Rental Vehicle Excess Insurance,7,LATVIA,NaN,11.88,43,No
26917,CWT,Travel Agency,Online,Rental Vehicle Excess Insurance,2,AZERBAIJAN,NaN,11.88,31,No
27058,LWC,Travel Agency,Online,Single Trip Travel Protect Silver,12,ARGENTINA,NaN,14.75,28,No
33127,JZI,Airlines,Online,Value Plan,7,MACAO,NaN,12.60,40,No
38534,EPX,Travel Agency,Online,1 way Comprehensive Plan,79,EGYPT,NaN,0.00,36,No


Jumlah `Net Sales` yang masih Nan: 9


Terlihat masih ada nilai 0 sebanyak 9 baris. Kita akan imput sisa nilai null ini menggunakan `Iterative Imputer` pada tahap selanjutnya setelah imputasi menggunakan `FunctionTransformer` di pipeline nantinya.

Selanjutnya kita akan tangani missing value pada fitur `Commission (in value)`.

**`Commision (in value)`**

Jika dilihat dari penjelasannya, fitur ini seharusnya tidak berkorelasi dengan keputusan orang untuk claim atau tidak. Fitur ini hanya komisi yang didapat agen dari hasil penjualan produk saja, dan lebih berhubungan sama penghasilan si agen.

Coba kita lihat proporsi nilai 0 pada fitur ini.

In [24]:
df_travel['Commision (in value)'].isnull().sum() / len(df_travel) * 100

np.float64(0.0)

Proporsi nilai 0 (missing value)-nya juga besar yang mana lebih dari 50%. Asumsi sementara kolom ini akan di drop saja, namun kita akan pastikan pada saat tahap EDA nanti. Jika memang akan digunakan, maka akan diimput menggunakan `Iterative Imputer`.

Selanjutnya menangani anomali pada fitur `Age`.

**`Age`**

In [25]:
# Lihat data dengan 'Age' = 0
df_travel[df_travel['Age']==0]

,Agency,Agency Type,Distribution Channel,Product Name,Duration,Destination,Net Sales,Commision (in value),Age,Claim
37815,TST,Travel Agency,Offline,Travel Cruise Protect,94,THAILAND,30.0,10.5,0,No
44319,TST,Travel Agency,Offline,Travel Cruise Protect,94,THAILAND,30.0,10.5,0,No


Ternyata hanya 2 data dengan umur = 0, dimana produk yang terjual sama dari agen yang sama dengan durasi dan tujuan yang sama. Kita akan coba lihat data lain dari keempat fitur yang sama tersebut untuk melihat transaksi ini hanya terjadi 2 kali ini saja atau ada data lain yang umurnya tidak 0.

In [26]:
# Cek nilai pada Agen, produk, durasi, dan tujuan yang sama dengan 'Age' = 0
df_travel[(df_travel['Agency']=='TST') & (df_travel['Product Name'] == 'Travel Cruise Protect') & (df_travel['Duration']==94) & (df_travel['Destination'] == 'THAILAND')]

,Agency,Agency Type,Distribution Channel,Product Name,Duration,Destination,Net Sales,Commision (in value),Age,Claim
19990,TST,Travel Agency,Offline,Travel Cruise Protect,94,THAILAND,30.0,10.5,52,No
28019,TST,Travel Agency,Offline,Travel Cruise Protect,94,THAILAND,30.0,10.5,55,No
29047,TST,Travel Agency,Offline,Travel Cruise Protect,94,THAILAND,30.0,10.5,55,No
29896,TST,Travel Agency,Offline,Travel Cruise Protect,94,THAILAND,30.0,10.5,48,No
31105,TST,Travel Agency,Offline,Travel Cruise Protect,94,THAILAND,30.0,10.5,35,No
37815,TST,Travel Agency,Offline,Travel Cruise Protect,94,THAILAND,30.0,10.5,0,No
44319,TST,Travel Agency,Offline,Travel Cruise Protect,94,THAILAND,30.0,10.5,0,No


Ternyata ada transaksi lain pada fitur-fitur yang sama tersebut. Dapat kita simpulkan bahwa nilai 0 ini kemungkinan besar adalah salah input / error / umur memang tidak diketahui pada saat input. Maka dari itu kita akan coba input nilainya dengan rata-rata umur dari transaksi yang sama tersebut, agar data semakin bervariasi namun masih dalam rentang umur pada transaksi yang sama.

Karena datanya hanya 2, maka akan kita imput langsung saja tidak menggunakan pipeline, karena dampaknya tidak akan terlalu besar untuk model jika terjadi kebocoran informasi data.

In [27]:
# Referensi rata-rata umur dari Agen, Produk, Durasi, dan Destination yang sama (tanpa umur 0)
ref_mean_age = df_travel[(df_travel['Agency'] == 'TST') 
                         & (df_travel['Product Name'] == 'Travel Cruise Protect') 
                         & (df_travel['Duration']==94) 
                         & (df_travel['Destination'] == 'THAILAND') 
                         & (df_travel['Age'] != 0)]['Age'].mean()

# Mengisi nilai 0 dengan rata-rata umurnya
df_travel.loc[(df_travel['Agency'] == 'TST') 
              & (df_travel['Product Name'] == 'Travel Cruise Protect') 
              & (df_travel['Duration']==94) & (df_travel['Destination'] == 'THAILAND') 
              & (df_travel['Age'] == 0), 'Age'] = ref_mean_age

# Cek sekali lagi untuk umur = 0
print(f'Jumlah data pada fitur umur yang masih 0: {df_travel[df_travel["Age"] == 0].shape[0]}')

Jumlah data pada fitur umur yang masih 0: 0


Kita sudah berhasil mengganti nilai 0 pada `Age`. 

Proses deteksi missing value dan cara-cara pengisiannya sudah selesai dan ditentukan, maka kesimpulan dari tahap ini adalah:

- Terdapat nilai null pada fitur `Gender` dengan persentase 71.39% dari total data di fitur tersebut, sehingga kita putuskan untuk menghapusnya.
- Terdapat nilai yang anomali pada 4 fitur numerik yaitu `Duration`, `Net Sales`, `Commission (in value)`, dan `Age`. Nilai anomali tersebut kita perlakukan sebagai missing value, dimana nilainya kita ganti menjadi nilai yang lebih wajar.
- Nilai anomali pada fitur `Duration` berupa angka minus dan nol (0). Nilai negatifnya berhasil kita rubah menjadi nilai positif dan nilai nolnya (0) kita buatkan `FunctionTransformer` yang proses pengisiannya nanti akan dimasukkan ke dalam tahapan pipeline. Sedangkan untuk 3 baris sisa data 0 kita hapus karena tidak ditemukan kemiripan dengan transaksi lain, dan karena hanya sedikit sehingga tidak akan berpengaruh signifikan pada model nantinya.
- Hal yang sama juga kita lakukan pada fitur `Net Sales` karena terdapat nilai anomali yang mirip dengan `Duration`. Namun nilai nol disini terlebih dahulu kita rubah menjadi Nan karena tipe data ini adalah float, dan nilai yang akan diisikan adalah nilai aktual hasil dari kombinasi fitur `Product Name` dan `Destination`. Pengisiannya melibatkan 2 tahapan pada pipeline nantinya, yaitu `FunctionTransformer` yang sudah dibuat, dan juga `IterativeImputer` untuk sisa nilai Nan-nya. Tidak ada baris data yang kita hapus disini.
- Untuk fitur `Commission (in value)` kita asumsikan tidak akan berpengaruh terhadap target, karena fitur ini tidak langsung berhubungan dengan nasabah melainkan kepada agen yang menjual produk tersebut. Namun kita akan lihat lebih lanjut pada hasil EDA, jika memang tidak berpengaruh maka kita akan buang saja, dan jika berpengaruh maka akan kita imputasi menggunakan `IterativeImputer`
- Sedangkan untuk fitur `Age` hanya terdapat nilai anomali berupa (0), jadi kita ganti saja nilai 0 ke nilai yang wajarnya. Dimana nilainya kita ganti dengan rata-ratanya pada setiap transaksi yang sama.

## **EXPLORATORY DATA ANALYSIS (EDA)**

## **DATA PREPARATION**

### 1. Define Feature (X) & Target (y)

### 2. Data Splitting

### 3. Preprocessing

## **MODELLING & EVALUATION**

### 1. Cross Validation

### 2. Hyperparameter Tuning

### 3. Prediction

### 4. Evaluation